In [1]:

!pip install openai PyMuPDF pandas openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 45.5 MB/s eta 0:00:00


In [2]:
import fitz  # PyMuPDF
import os
import json
import pandas as pd
from openai import OpenAI
from google.colab import files

In [ ]:
# Upload OpenAI API Key
from getpass import getpass
OPENAI_API_KEY = getpass('Enter API Key: ')
client = OpenAI(api_key=OPENAI_API_KEY)

In [ ]:
# Upload CVs (Multiple) and JD (Single) PDFs
print(" Upload CV PDFs (Multiple Allowed)")
cv_files = files.upload()

print(" Upload a single JD (Job Description) PDF")
jd_file = files.upload()

 Upload CV PDFs (Multiple Allowed)


Saving , resume - Haya Khan.pdf to , resume - Haya Khan.pdf
Saving 5-a - Muhammad Ali.pdf to 5-a - Muhammad Ali.pdf
Saving 2022274-3 - Malik Abdullah Awan.pdf to 2022274-3 - Malik Abdullah Awan.pdf
Saving AbdullahMustafaDAResume - Candide.pdf to AbdullahMustafaDAResume - Candide.pdf
Saving ABIA KHAN_2 - Abia Khan.pdf to ABIA KHAN_2 - Abia Khan.pdf
Saving Abrar_Resume - IBRAR UL HAQ.pdf to Abrar_Resume - IBRAR UL HAQ.pdf
Saving AfaqAhmad_Resume.pdf - Afaq Ahmad.pdf to AfaqAhmad_Resume.pdf - Afaq Ahmad.pdf
Saving ali_AI_CV - Ali Faisal Khan.pdf to ali_AI_CV - Ali Faisal Khan.pdf
Saving Alihassanresume (1)-1 - Alihassan Abbasi.pdf to Alihassanresume (1)-1 - Alihassan Abbasi.pdf
Saving AliRaza_CV - Ali Raza.pdf to AliRaza_CV - Ali Raza.pdf
Saving Amiya Aden Balouch - Amiya Aden Balouch.pdf to Amiya Aden Balouch - Amiya Aden Balouch.pdf
Saving Bilal-resume - Bilal Aslam..pdf to Bilal-resume - Bilal Aslam..pdf
 Upload a single JD (Job Description) PDF


Saving Position.pdf to Position (1).pdf


In [ ]:
# Extract Text from CVs
cv_data = []
for filename, content in cv_files.items():
    with open(filename, "wb") as f:
        f.write(content)
    doc = fitz.open(filename)
    text = "".join(page.get_text() for page in doc)
    cv_data.append({"filename": filename, "content": text})

In [ ]:
# Extract Text from JD
jd_filename, jd_content = list(jd_file.items())[0]
with open(jd_filename, "wb") as f:
    f.write(jd_content)
doc = fitz.open(jd_filename)
jd_text = "".join(page.get_text() for page in doc)

In [ ]:
# Normalize Job Description with GPT (Corrected for JSON response format)
position_structure = {
    "JobTitle": "", "Department": "", "ReportsTo": "", "JobPurpose": "",
    "KeyResponsibilities": [], "WorkModality": "", "AdditionalDetails": "",
    "Qualifications": {
        "education": [], "work_experience": [], "skills": [],
        "certifications_and_courses": [], "languages": [{"language": "", "profieciency": ""}],
        "systems_knowledge": []
    }
}
position_json = json.dumps(position_structure)

jd_response = client.chat.completions.create(
    model="gpt-4o",
    response_format={"type": "json_object"},
    messages=[
        {
            "role": "system",
            "content": "You are a recruiter assistant that converts job descriptions into structured JSON format for analysis."
        },
        {
            "role": "user",
            "content": f"Given the following job description, extract and return the information using this JSON structure:\n\n{position_json}\n\nJob Description:\n{jd_text}"
        }
    ]
)

structured_jd = jd_response.choices[0].message.content


In [ ]:
# ✅ Evaluate CVs Against Job Description (Corrected for JSON output)
evaluation_structure = {
    "dimensions": [
        {"name": "education", "value": 0, "reasoning": ""},
        {"name": "work_experience", "value": 0, "reasoning": ""},
        {"name": "skills", "value": 0, "reasoning": ""},
        {"name": "certifications_and_courses", "value": 0, "reasoning": ""},
        {"name": "languages", "value": 0, "reasoning": ""},
        {"name": "Technology_knowledge", "value": 0, "reasoning": ""},
        {"name": "general_assessment", "value": 0, "reasoning": ""}
    ]
}
evaluation_json = json.dumps(evaluation_structure)

# Create dataframe to store final output
results_df = pd.DataFrame()

for cv in cv_data:
    cv_filename = cv["filename"]
    cv_text = cv["content"]

    response = client.chat.completions.create(
        model="gpt-4o",
        response_format={"type": "json_object"},
        messages=[
            {
                "role": "system",
                "content": "You are a professional recruiter who evaluates candidates based on job descriptions using structured JSON output."
            },
            {
                "role": "user",
                "content": f"""Evaluate the following candidate CV in JSON format.
Use this structure:
{evaluation_json}

Job Description:
{structured_jd}

Candidate CV:
{cv_text}

Please return your evaluation in the exact JSON format above, including a numeric score and an explanation for each dimension."""
            }
        ]
    )

    eval_data = json.loads(response.choices[0].message.content)
    row = {"filename": cv_filename}
    for dim in eval_data["dimensions"]:
        row[f"{dim['name']}_score"] = dim["value"]
        row[f"{dim['name']}_explanation"] = dim["reasoning"]
    results_df = pd.concat([results_df, pd.DataFrame([row])], ignore_index=True)
    print(f"✔ Evaluated: {cv_filename}")


✔ Evaluated: , resume - Haya Khan.pdf
✔ Evaluated: 5-a - Muhammad Ali.pdf
✔ Evaluated: 2022274-3 - Malik Abdullah Awan.pdf
✔ Evaluated: AbdullahMustafaDAResume - Candide.pdf
✔ Evaluated: ABIA KHAN_2 - Abia Khan.pdf
✔ Evaluated: Abrar_Resume - IBRAR UL HAQ.pdf
✔ Evaluated: AfaqAhmad_Resume.pdf - Afaq Ahmad.pdf
✔ Evaluated: ali_AI_CV - Ali Faisal Khan.pdf
✔ Evaluated: Alihassanresume (1)-1 - Alihassan Abbasi.pdf
✔ Evaluated: AliRaza_CV - Ali Raza.pdf
✔ Evaluated: Amiya Aden Balouch - Amiya Aden Balouch.pdf
✔ Evaluated: Bilal-resume - Bilal Aslam..pdf


In [ ]:
# Export to Excel
excel_file = "GPDA_CV_Evaluation_Results.xlsx"
results_df.to_excel(excel_file, index=False)
files.download(excel_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>